## Train a character-level GPT on some text data

The inputs here are simple text files, which we chop up to individual characters and then train GPT on. So you could say this is a char-transformer instead of a char-rnn. Doesn't quite roll off the tongue as well. In this example we will feed it some Shakespeare, which we'll get it to predict character-level.

In [1]:
from allennlp.data.tokenizers import Token, Tokenizer, PretrainedTransformerTokenizer

import nltk
import numpy as np
from os import listdir
from os.path import join as pathjoin
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import tqdm

from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig
# make deterministic
from mingpt.utils import sample, set_seed
set_seed(42)
np.random.seed(42)

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
transformer_model = 'bert-base-cased'

In [3]:
import math
from torch.utils.data import Dataset


def detokenize(tokens):
    return ' '.join([str(x) for x in tokens[1:-1]]).replace(' ##', '')

class BPEDataset(Dataset):
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

In [4]:
block_size = 128
tokenizer = PretrainedTransformerTokenizer(transformer_model)

In [5]:
from mingpt.utils import sample


def generate_dataset(train_text_file, state_dict_file, n_layer=4, n_head=4, n_embd=256,
                     texts_count=100, text_len=500):
    text_sentences = nltk.tokenize.sent_tokenize(open(train_text_file, 'r').read())
    tokens = np.concatenate([tokenizer.tokenize(sent) for sent in text_sentences])
    bos_token = tokens[0]
    print(bos_token)
    tokens = [str(token) for token in tokens]
    train_dataset = BPEDataset(tokens, block_size) 
    mconf = GPTConfig(
        train_dataset.vocab_size, train_dataset.block_size,
        n_layer=n_layer, n_head=n_head, n_embd=n_embd
    )
    model = GPT(mconf)
    model.load_state_dict(torch.load(state_dict_file))
    print("model is loaded")
    tconf = TrainerConfig(num_workers=1)
    trainer = Trainer(model, train_dataset, None, tconf)
    
    for text_id in range(texts_count):
        context = [train_dataset.itos[np.random.randint(train_dataset.vocab_size)]]
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
        y = sample(model, x, text_len, temperature=1.0, sample=True, top_k=10)[0]
        completion = ' '.join([train_dataset.itos[int(i)] for i in y]).replace(' ##', '')
        completion = completion.replace('[SEP]', '').replace('[CLS]', '')
        yield completion

In [6]:
GENRE_DATA_DIR = '/home/mlepekhin/data/genre'
GPT_MODELS_DIR = '/home/mlepekhin/models/mini_gpt_bpe_tuned/'
LANG = 'en'

In [7]:
result_df = pd.DataFrame()

In [8]:
for train_text_file in tqdm.tqdm(listdir(pathjoin(GENRE_DATA_DIR, LANG))):
    label = train_text_file[:-4]
    print(label)
    for text in generate_dataset(
        pathjoin(GENRE_DATA_DIR, LANG, train_text_file),
        pathjoin(GPT_MODELS_DIR, LANG, label)
    ):
        result_df = result_df.append({'text': text, 'target': label}, ignore_index=True)

  0%|          | 0/11 [00:00<?, ?it/s]

A4
[CLS]
data has 208998 characters, 11296 unique.
model is loaded


  9%|▉         | 1/11 [09:02<1:30:27, 542.71s/it]

A7
[CLS]
data has 162210 characters, 9736 unique.
model is loaded


 18%|█▊        | 2/11 [17:44<1:20:28, 536.47s/it]

A14
[CLS]
data has 396230 characters, 12440 unique.


 18%|█▊        | 2/11 [17:58<1:20:53, 539.25s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/home/mlepekhin/models/mini_gpt_bpe_tuned/en/A14'

In [ ]:
result_df.head()

In [ ]:
result_df.to_csv('/home/mlepekhin/data/min_gpt_bpe/en_tuned')

In [ ]:
df = pd.read_csv('/home/mlepekhin/data/en_test')
print('mean len', np.median([len(tokenizer.tokenize(sent)[1:-1]) for sent in df.text.values]))